In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from imdb import Cinemagoer

In [3]:
import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

In [4]:
from profanity_detector.get_data import movie_data, get_all_movie_data, run_and_display, get_all_movie_data

In [5]:
from profanity_detector.feature_data_enrichment import get_movie_location, get_set_location, get_coordinates, enrich_locations
from geopy.geocoders import Nominatim

In [6]:
import nltk

In [7]:
import spacy

In [8]:
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
nltk.downloader.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 16.7 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Location tagger

In [10]:
import locationtagger

In [116]:
movie_meta, quotes_df, reviews_df, locations_df = movie_data('lord of the rings')

In [117]:
location = locations_df['locations'][2]

In [118]:
location

'Kaitoke Regional Park, Wellington Region, New Zealand::(Rivendell) (River Anduin)'

# preparing the data

In [14]:
def get_movie_location(location):
    if '::' in location:
        splitlist = location.split('::')
        movie_location = splitlist[1]
            
    else:
        movie_location = ''
           
    return movie_location

In [15]:
def get_set_location(location):
    if '::' in location:
        splitlist = location.split('::')
        set_location = splitlist[0]
    
    else:
        set_location = location
    
    
    return set_location 

In [16]:
locations_df['movie_location'] = locations_df.apply(lambda x: get_movie_location(x['locations']),
                                                                     axis=1 )

In [17]:
locations_df['set_location'] = locations_df.apply(lambda x: get_set_location(x['locations']),
                                                                     axis=1 )

In [71]:
locations_df

,locations,imdb_id,title,movie_location,set_location,latitude,longitude
0,"St. Mary's Church - 203 E. 10th St. Austin, Te...",0985694,Machete,,"St. Mary's Church - 203 E. 10th St. Austin, Te...",NaN,NaN
1,"Austin, Texas, USA",0985694,Machete,,"Austin, Texas, USA",NaN,NaN
2,"Troublemaker Studios - 4900 Old Manor Road, Au...",0985694,Machete,(studio),"Troublemaker Studios - 4900 Old Manor Road, Au...",NaN,NaN
3,"Texas, USA",0985694,Machete,,"Texas, USA",NaN,NaN
4,USA,0985694,Machete,,USA,NaN,NaN


In [19]:
locations_df['set_location'][2]

'Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA'

# geograpy

In [20]:
import geograpy

In [21]:
from geopy.geocoders import Nominatim

In [22]:
! geograpy-nltk

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tomershtein/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


value_when_true if condition, else value_when_false

In [23]:
loc_list = list(locations_df['set_location'])

In [24]:
loc_list

["St. Mary's Church - 203 E. 10th St. Austin, Texas, USA",
 'Austin, Texas, USA',
 'Troublemaker Studios - 4900 Old Manor Road, Austin, Texas, USA',
 'Texas, USA',
 'USA']

In [75]:
locations_df = locations_df.drop(['latitude','longitude'], axis=1)

In [121]:
locations_df

,locations,imdb_id,title,movie_location,set_location
0,"Wellington, New Zealand::(Bree)",0120737,The Lord of the Rings: The Fellowship of the Ring,(Bree),"Wellington, New Zealand"
1,"Hinuera Valley, Matamata, Waikato, New Zealand...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Hobbiton),"Hinuera Valley, Matamata, Waikato, New Zealand"
2,"Kaitoke Regional Park, Wellington Region, New ...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Rivendell) (River Anduin),"Kaitoke Regional Park, Wellington Region, New ..."
3,"Queenstown, Otago, New Zealand::(Ford of Bruin...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Ford of Bruinen) (River Anduin) (Amon Hen),"Queenstown, Otago, New Zealand"
4,"Milford Sound, Fiordland National Park, Southl...",0120737,The Lord of the Rings: The Fellowship of the Ring,,"Milford Sound, Fiordland National Park, Southl..."
5,"Port Waikato, Waikato, New Zealand::(Weathertop)",0120737,The Lord of the Rings: The Fellowship of the Ring,(Weathertop),"Port Waikato, Waikato, New Zealand"
6,"Fernside, Featherston, Wairarapa, New Zealand:...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Lothlorien),"Fernside, Featherston, Wairarapa, New Zealand"
7,"Rangipo Desert, Waiouru, Tongariro National Pa...",0120737,The Lord of the Rings: The Fellowship of the Ring,,"Rangipo Desert, Waiouru, Tongariro National Pa..."
8,"Rangitikei River, Manawatu-Wanganui, New Zeala...",0120737,The Lord of the Rings: The Fellowship of the Ring,(River Anduin),"Rangitikei River, Manawatu-Wanganui, New Zealand"
9,"Tongariro National Park, Central Plateau, New ...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Mordor),"Tongariro National Park, Central Plateau, New ..."


In [122]:
for location in locations_df['set_location']:
    print(location, get_coordinates(location))    

Wellington, New Zealand (-41.2887953, 174.7772114)
Hinuera Valley, Matamata, Waikato, New Zealand (-37.7741407, 175.4883742)
Kaitoke Regional Park, Wellington Region, New Zealand (-41.0495879, 175.1791197)
Queenstown, Otago, New Zealand (-45.0321923, 168.661)
Milford Sound, Fiordland National Park, Southland, New Zealand (-44.67184, 167.9254592)
Port Waikato, Waikato, New Zealand (-37.3913264, 174.717334)
Fernside, Featherston, Wairarapa, New Zealand (-41.0885537, 175.3505731)
Rangipo Desert, Waiouru, Tongariro National Park, Central Plateau, New Zealand (-39.4772125, 175.6697752)
Rangitikei River, Manawatu-Wanganui, New Zealand (-39.7665195, 175.8152587)
Tongariro National Park, Central Plateau, New Zealand (-39.1727294, 175.4037274)
Lake Wakatipu, Queenstown, Otago, New Zealand (-45.08880415, 168.52111077909876)
Mavora Lakes, Southland, New Zealand (-45.36653, 168.11838)
Tarras, Wanaka, Otago, New Zealand (-44.6941691, 169.1364637)
Canterbury, New Zealand (-43.49417615, 171.809844758

AttributeError: 'NoneType' object has no attribute 'latitude'

In [124]:
loc = Nominatim(user_agent="profanity_detector")

In [125]:
getLoc = loc.geocode('Lake Alta, The Remarkables, Nelson, South Island, New Zealand')

getLoc is None

In [127]:
places = geograpy.get_place_context(text='Lake Alta, The Remarkables, Nelson, South Island, New Zealand')

In [128]:
places.__dict__

{'debug': False,
 'correctMisspelling': False,
 'storageConfig': <lodstorage.storageconfig.StorageConfig at 0x7f229458b978>,
 'db_path': '/home/tomershtein/.geograpy3',
 'db_file': '/home/tomershtein/.geograpy3/locations.db',
 'view': 'CityLookup',
 'sqlDB': <lodstorage.sql.SQLDB at 0x7f229422e160>,
 'aliases': {'UK': 'GB',
  'USA': 'United States of America',
  'United States': 'United States of America'},
 'dbVersion': '2021-08-18 16:15:00',
 'places': ['Lake',
  'Alta',
  'Remarkables',
  'Nelson',
  'South Island',
  'New Zealand'],
 'country_mentions': [('New Zealand', 1),
  ('Norway', 1),
  ('United States of America', 1),
  ('Australia', 1),
  ('Germany', 1),
  ('Canada', 1),
  ('United Kingdom', 1),
  ('Argentina', 1)],
 'countries': ['New Zealand',
  'Norway',
  'United States of America',
  'Australia',
  'Germany',
  'Canada',
  'United Kingdom',
  'Argentina'],
 'country_regions': {'New Zealand': ['South Island', 'Nelson']},
 'region_mentions': [('Nelson', 1), ('South Islan

In [138]:
country = places.countries[0] if places.countries != [] else ''

#extract region
if places.regions != [] and places.regions[0] != places.countries[0]:
    region = places.regions[0]  
elif places.regions != [] and places.regions[0] == places.countries[0] and len(places.regions) > 1:
     region = places.regions[1]
else:
    region = ''    

#extract city
city = places.cities[0] if places.cities != [] else ''

#create alt_location string
loc_parse = [city, region, country]
place = []
for l in loc_parse:
    if l != '':
        place.append(l) 
alt_location = ', '.join(place)

In [139]:
alt_location

'Nelson, South Island, New Zealand'

In [141]:
alt_list = alt_location.split(',')

alt_list.remove(alt_list[0])

alt_location = ', '.join(alt_list)

alt_location

In [148]:
getLoc2 = loc.geocode(' South Island,  New Zealand')

getLoc2 is None

False

In [178]:
getLoc2.raw

{'place_id': 282866552,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 3986157,
 'boundingbox': ['-46.6754988', '-40.4978624', '166.4255531', '174.315863'],
 'lat': '-43.586562',
 'lon': '171.21232939964904',
 'display_name': 'South Island, New Zealand / Aotearoa',
 'class': 'place',
 'type': 'island',
 'importance': 0.9734959443839385}

In [ ]:
locations_df['country'] = locations_df.apply(lambda x: , axis=1)

In [ ]:
def get_country_from_coordinates(latitude, longitude)
    loc = Nominatim(user_agent="profanity_detector")
    reverse_location = loc.reverse([getLoc2.latitude, getLoc2.longitude])
    country_name = reverse_location.raw['address']['country']
    return country_name

In [183]:
reverse_location = loc.reverse([getLoc2.latitude, getLoc2.longitude])

In [185]:
reverse_location.raw['address']['country']

'New Zealand / Aotearoa'

In [155]:
locations_df.iloc[:34,:]

,locations,imdb_id,title,movie_location,set_location
0,"Wellington, New Zealand::(Bree)",0120737,The Lord of the Rings: The Fellowship of the Ring,(Bree),"Wellington, New Zealand"
1,"Hinuera Valley, Matamata, Waikato, New Zealand...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Hobbiton),"Hinuera Valley, Matamata, Waikato, New Zealand"
2,"Kaitoke Regional Park, Wellington Region, New ...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Rivendell) (River Anduin),"Kaitoke Regional Park, Wellington Region, New ..."
3,"Queenstown, Otago, New Zealand::(Ford of Bruin...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Ford of Bruinen) (River Anduin) (Amon Hen),"Queenstown, Otago, New Zealand"
4,"Milford Sound, Fiordland National Park, Southl...",0120737,The Lord of the Rings: The Fellowship of the Ring,,"Milford Sound, Fiordland National Park, Southl..."
5,"Port Waikato, Waikato, New Zealand::(Weathertop)",0120737,The Lord of the Rings: The Fellowship of the Ring,(Weathertop),"Port Waikato, Waikato, New Zealand"
6,"Fernside, Featherston, Wairarapa, New Zealand:...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Lothlorien),"Fernside, Featherston, Wairarapa, New Zealand"
7,"Rangipo Desert, Waiouru, Tongariro National Pa...",0120737,The Lord of the Rings: The Fellowship of the Ring,,"Rangipo Desert, Waiouru, Tongariro National Pa..."
8,"Rangitikei River, Manawatu-Wanganui, New Zeala...",0120737,The Lord of the Rings: The Fellowship of the Ring,(River Anduin),"Rangitikei River, Manawatu-Wanganui, New Zealand"
9,"Tongariro National Park, Central Plateau, New ...",0120737,The Lord of the Rings: The Fellowship of the Ring,(Mordor),"Tongariro National Park, Central Plateau, New ..."


In [160]:
for location in locations_df['set_location'][0:34]:
    print(location, get_coordinates(location)) 

Wellington, New Zealand (-41.2887953, 174.7772114)
Hinuera Valley, Matamata, Waikato, New Zealand (-37.7741407, 175.4883742)
Kaitoke Regional Park, Wellington Region, New Zealand (-41.0495879, 175.1791197)
Queenstown, Otago, New Zealand (-45.0321923, 168.661)
Milford Sound, Fiordland National Park, Southland, New Zealand (-44.67184, 167.9254592)
Port Waikato, Waikato, New Zealand (-37.3913264, 174.717334)
Fernside, Featherston, Wairarapa, New Zealand (-41.0885537, 175.3505731)
Rangipo Desert, Waiouru, Tongariro National Park, Central Plateau, New Zealand (-39.4772125, 175.6697752)
Rangitikei River, Manawatu-Wanganui, New Zealand (-39.7665195, 175.8152587)
Tongariro National Park, Central Plateau, New Zealand (-39.1727294, 175.4037274)
Lake Wakatipu, Queenstown, Otago, New Zealand (-45.08880415, 168.52111077909876)
Mavora Lakes, Southland, New Zealand (-45.36653, 168.11838)
Tarras, Wanaka, Otago, New Zealand (-44.6941691, 169.1364637)
Canterbury, New Zealand (-43.49417615, 171.809844758

In [30]:
location_df2['movie_location'] = location_df2.apply(lambda x: get_movie_location(x['locations']), axis=1)

In [31]:
location_df2['set_location'] = location_df2.apply(lambda x: get_set_location(x['locations']), axis=1)

In [69]:
location_df2[['latitude','longitude']] = location_df2.apply(lambda x: get_coordinates(x['set_location']), axis=1, result_type='expand')

In [ ]:
enrich_locations(locations_df)

# Folium

In [88]:
locations_df.iloc[:1,:]

,locations,imdb_id,title,movie_location,set_location,latitude,longitude
0,"Nashville, Tennessee, USA::(exterior scenes: s...",0133093,The Matrix,(exterior scenes: skyline in opening Trinity r...,"Nashville, Tennessee, USA",36.162277,-86.774298


In [89]:
len(locations_df['latitude'])

22

In [105]:
list(locations_df[['latitude', 'longitude']].loc[0])

[36.1622767, -86.7742984]

In [106]:
locations = locations_df[['latitude', 'longitude']]

In [107]:
locationlist = locations.values.tolist()

In [108]:
locationlist

[[36.1622767, -86.7742984],
 [-33.8548157, 151.2164539],
 [-33.8931044, 151.2040292],
 [37.7790262, -122.419906],
 [-33.8548157, 151.2164539],
 [-33.8548157, 151.2164539],
 [-33.8548157, 151.2164539],
 [-33.890960899999996, 151.22750517838074],
 [-33.8548157, 151.2164539],
 [-33.86376205, 151.20362772900535],
 [-33.8548157, 151.2164539],
 [-33.8548157, 151.2164539],
 [-33.8573904, 151.2030854],
 [-33.8645002, 151.1743543],
 [-33.9002762, 151.2073137],
 [-33.8548157, 151.2164539],
 [-33.8548157, 151.2164539],
 [-33.8548157, 151.2164539],
 [-31.8759835, 147.2869493],
 [-33.8845123, 151.2100301],
 [-33.8548157, 151.2164539],
 [-33.8863291, 151.1998211]]

In [109]:
locationlist[0]

[36.1622767, -86.7742984]

In [112]:
import folium

In [115]:
map = folium.Map(location=locationlist[0], zoom_start=12)
for point in range(0, len(locations_df['latitude'])):
    folium.Marker(locationlist[point], popup=locations_df['movie_location'][point]).add_to(map)
map